In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# import the trainingset
data = pd.read_csv('letter-recognition.csv',header=None)  
data.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,T,2,8,3,5,1,8,13,0,6,6,10,8,0,8,0,8
1,I,5,12,3,7,2,10,5,5,4,13,3,9,2,8,4,10
2,D,4,11,6,8,6,10,6,2,6,10,3,7,3,7,3,9
3,N,7,11,6,6,3,5,9,4,6,4,4,10,6,10,2,8
4,G,2,1,3,1,1,8,6,6,6,6,5,9,1,7,5,10


In [10]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
import scipy.stats as stats
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KernelDensity as KDe
from sklearn.ensemble import GradientBoostingRegressor

In [48]:
# prepare the trainging data and test data
data = pd.read_csv('letter-recognition.csv',header=None) 
#X = data[data.columns[1:]].values
#y = data[0].values
#print(X)
#print(y)
#cols = data.columns.tolist()
#data.drop_duplicates(subset=cols[:-1], inplace=True)

# pair H and K
data_h = data[data[0] == 'H']
data_k = data[data[0] == 'K']
data1 = pd.concat([data_h, data_k], axis=0)
data1.loc[data1[0] == 'H', 0] = 0
data1.loc[data1[0] == 'K', 0] = 1 

imputer = KNNImputer(n_neighbors=1)
X = imputer.fit_transform(data1)

print(X)
cv = KFold(n_splits=10, random_state=1, shuffle=True)

[[ 0.  4.  5. ...  8.  3.  8.]
 [ 0.  6.  9. ...  8.  4.  8.]
 [ 0.  3.  3. ...  8.  3.  7.]
 ...
 [ 1.  7. 14. ...  7.  4.  8.]
 [ 1.  3.  1. ...  8.  5.  9.]
 [ 1.  5. 11. ...  8.  2. 11.]]


In [49]:
# cross validation
def cross_validation(model):
    in_sample_error = []
    out_sample_error = []
    err_zscore = []

    for train_index, val_index in cv.split(X):
        Xtrain, Xval = X[train_index], X[val_index]
        ytrain, yval = y[train_index], y[val_index]
        model.fit(Xtrain,ytrain)

        pred_train,pred_val = model.predict(Xtrain),model.predict(Xval)
        pred = np.concatenate((pred_train, pred_val), axis=None)
        true_label = np.concatenate((ytrain,yval),axis=None)
        error = np.square(pred - true_label)/true_label.shape[0]
        #error = pred - true_label
        err_zscore.append(stats.zscore(error).reshape(-1,1))
        in_sample_error.append(mean_squared_error(pred_train, ytrain, squared=True))
        out_sample_error.append(mean_squared_error(pred_val, yval, squared=True))

    in_sample_error = np.array(in_sample_error)
    out_sample_error = np.array(out_sample_error)
    return in_sample_error,out_sample_error, err_zscore


* Random Forest

In [51]:
RF = RandomForestRegressor(max_depth=4, random_state=0)
RF_in_sample_err,RF_out_sample_err, RF_zscore_err=cross_validation(RF)
# print(np.mean(RF_out_sample_err))
kde = KDe(bandwidth=1.0, kernel='gaussian')
kde.fit(RF_zscore_err)
lp = kde.score_samples(RF_zscore_err)
print(lp.shape)
sns.kdeplot(RF_zscore_err[:,0])

ValueError: could not convert string to float: 'T'